In [10]:
import os
import time
import numpy as np
import seqdatasets
from eugene import preprocess as pp
from eugene import dataload as dl
from eugene import train
from eugene import models
import seqdata as sd
import xarray as xr
from eugene import settings

In [11]:
# Change this to where you would like to save all your results
import os
os.chdir("/cellar/users/aklie/projects/ML4GLand/issues/41")  # TODO: change this to your own directory
cwd = os.getcwd()
cwd

'/cellar/users/aklie/projects/ML4GLand/issues/41'

In [2]:
sdata_train = seqdatasets.deAlmeida22("train")
sdata_val = seqdatasets.deAlmeida22("val")
pp.ohe_seqs_sdata(sdata_train)
pp.ohe_seqs_sdata(sdata_val)

Path /cellar/users/aklie/eugene_data/deAlmeida22 does not exist, creating new folder.
Finished downloading Sequences_Train.fa?download=1
Finished downloading Sequences_activity_Train.txt?download=1
Zarr file not found. Creating new zarr file.


100%|██████████| 402296/402296 [00:04<00:00, 85113.96it/s]


Finished downloading Sequences_Val.fa?download=1
Finished downloading Sequences_activity_Val.txt?download=1
Zarr file not found. Creating new zarr file.


100%|██████████| 40570/40570 [00:00<00:00, 86336.93it/s]


In [3]:
from eugene.models.zoo import DeepSTARR

In [4]:
arch = DeepSTARR(
    input_len=249, 
    output_dim=2, 
)
models.init_weights(arch)

In [5]:
model = models.SequenceModule(
    arch=arch,
    task="regression",
    loss_fxn="mse",
    optimizer="adam",
    optimizer_lr=0.002,
    optimizer_kwargs=dict(weight_decay=1e-6)
)

In [6]:
sdata_train["target"] = xr.concat([sdata_train[target_var] for target_var in ['Dev_log2_enrichment_quantile_normalized', 'Hk_log2_enrichment_quantile_normalized']], dim="_targets").transpose("_sequence", "_targets")
sdata_val["target"] = xr.concat([sdata_val[target_var] for target_var in ['Dev_log2_enrichment_quantile_normalized', 'Hk_log2_enrichment_quantile_normalized']], dim="_targets").transpose("_sequence", "_targets")

In [8]:
sdata_train.load(), sdata_val.load()

(<xarray.Dataset>
 Dimensions:                                  (_sequence: 402296, _length: 249,
                                               length: 249, _ohe: 4, _targets: 2)
 Coordinates:
   * _sequence                                (_sequence) object 'chr2L_5587_5...
 Dimensions without coordinates: _length, length, _ohe, _targets
 Data variables:
     seq                                      (_sequence, _length) |S1 b'\x80'...
     Dev_log2_enrichment                      (_sequence) float64 5.712 ... -2...
     Hk_log2_enrichment                       (_sequence) float64 1.363 ... -1...
     Dev_log2_enrichment_scaled               (_sequence) float64 3.527 ... -1.8
     Hk_log2_enrichment_scaled                (_sequence) float64 0.8735 ... -...
     Dev_log2_enrichment_quantile_normalized  (_sequence) float64 6.046 ... -2...
     Hk_log2_enrichment_quantile_normalized   (_sequence) float64 1.711 ... -1...
     ohe_seq                                  (_sequence, length, _oh

In [12]:
train_dl = sd.get_torch_dataloader(
    sdata_train,
    batch_size=128,
    sample_dims="_sequence",
    variables=["ohe_seq", "target"],
    shuffle=True,
    num_workers=4,
    prefetch_factor=2,
    transforms={"ohe_seq": lambda x: x.swapaxes(1, 2)}
)

val_dl = sd.get_torch_dataloader(
    sdata_val,
    batch_size=128,
    sample_dims="_sequence",
    variables=["ohe_seq", "target"],
    shuffle=False,
    num_workers=4,
    prefetch_factor=2,
    transforms={"ohe_seq": lambda x: x.swapaxes(1, 2)}
)

In [14]:
train.fit(
    model,
    train_dataloader=train_dl,
    val_dataloader=val_dl,
    gpus=1,
    epochs=100,
    early_stopping_patience=10,
    log_dir=settings.logging_dir,
    name="DeepSTARR",
    version="Trial7",
)

No seed set


/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib ...
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
